In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from joblib import Parallel, delayed
from keras.models import model_from_json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential


import constitutive
import displacement_solver
import gauss_pt_coord
import mesh_gen
import quadrature
import stress_gauss
import spr_stress
import patch_n_int_nodes
import stress_nodes_dc


2023-04-07 10:03:56.399146: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-07 10:03:56.405007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 10:03:56.532413: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-07 10:03:56.534603: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 10:04:01.176181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
# # 1) Inputs
# a) Problem parameters
# Domain geometry
domain_coord = np.array([[0, 0], [48, 44], [48, 60], [0, 44], [
                        24, 22], [48, 52], [24, 52], [0, 22]])
# Body force components
b = np.array([[0], [0]])
# Traction components
q = 1/16
T = np.array([[0, 0], [0, q], [0, 0]])
# b) Young's modulus
E = 1.0
# c) Poisson's ratio
nu = 1/3
# d) Problem type (0 ----> plane stress, 1 -----> plane strain)
problem_type = 0
# e) Element used for meshing (0 ---> 4 node quadrilateral, 1 -----> 8 node quadrilateral)
el_type_q4 = 0
el_type_q8 = 1
el_type_q9 = 2
# f) No. of Gauss points required for integration
ngp2d = 1
ngp1d = 2

# # g) Mesh sizes to be tested
N = [5]


In [4]:
# Q4 elements
u_list_q4 = Parallel(n_jobs=-1, verbose=100)(
    delayed(displacement_solver.solve_fem)(N[i], E, nu, ngp2d, ngp1d, el_type_q4, problem_type, domain_coord, b, T)
    for i in range(len(N)))

u_q4 = []
for i in range(len(N)):
    nx = N[i]
    ny = N[i]
    node_2d = np.arange((nx + 1) * (ny + 1)).reshape(nx + 1, ny + 1)
    node = node_2d[int(ny / 2), -1]
    dof = 2 * node + 1
    u_q4.append(u_list_q4[i][dof])
print(u_q4)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
(25, 8)
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.4s finished
[24.364007351422813]


In [5]:
ms=N[0]
u=u_list_q4[N.index(ms)]
nel=ms*ms  

In [6]:
C = constitutive.constitutive(E, nu, problem_type)
#nodal coordinates
nx = ms
ny = ms
el_type = 0
mesh_obj = mesh_gen.MeshGenerator(nx, ny, domain_coord.reshape(16, 1), el_type)
connect = mesh_obj.connectivity()
coord = mesh_obj.coord_array()


In [9]:
strs_ref=pd.read_csv('Data/ms256_ref_q9.csv')

In [10]:
X = pd.read_csv('Coord/mesh_256.csv')
Y = strs_ref;


In [11]:
# print(X);
# print(Y);

In [12]:
X = X.loc[:, ~X.columns.str.contains('^Unnamed')]
Y = Y.loc[:, ~Y.columns.str.contains('^Unnamed')]


In [13]:
# print(X);
# print(Y);

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
scaled_x=scaler.transform(X)
scaled_test=scaler.transform(coord)

/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [163]:
#ANN one model in whole domain
model1 = Sequential()
model1.add(Dense(units=50, activation='relu'))
model1.add(Dense(units=44, activation='relu'))
model1.add(Dense(units=37, activation='relu'))
model1.add(Dense(units=30, activation='relu'))
model1.add(Dense(units=24, activation='relu'))
model1.add(Dense(units=17, activation='relu'))
model1.add(Dense(units=10, activation='relu'))
model1.add(Dense(units=3, activation='relu'))
model1.add(Dense(units=3, activation='linear'))
model1.compile(
    loss='mse', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01))


In [164]:
# es = tf.keras.callbacks.EarlyStopping(
#     monitor='val_loss', mode='min', verbose=1, patience=50)


In [165]:
#Training from different optimizer
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', mode='min', verbose=1, patience=50)


In [166]:
# np.random.seed(1)
# history=model1.fit(x=scaled_x, y=Y, batch_size=256, epochs=500, verbose=1, validation_split=0.1,callbacks=[es])

In [167]:
#Model fitting for different optimizer
np.random.seed(1)
history=model1.fit(x=scaled_x, y=Y, batch_size=256, epochs=500, verbose=1, validation_split=0.1,callbacks=[es])

Epoch 1/500
233/233 [==============================] - 13s 28ms/step - loss: 0.0029 - val_loss: 0.0130
Epoch 2/500
233/233 [==============================] - 5s 22ms/step - loss: 0.0026 - val_loss: 0.0132
Epoch 3/500
233/233 [==============================] - 5s 23ms/step - loss: 0.0025 - val_loss: 0.0130
Epoch 4/500
233/233 [==============================] - 5s 21ms/step - loss: 0.0025 - val_loss: 0.0129
Epoch 5/500
233/233 [==============================] - 5s 22ms/step - loss: 0.0025 - val_loss: 0.0129
Epoch 6/500
233/233 [==============================] - 5s 21ms/step - loss: 0.0025 - val_loss: 0.0128
Epoch 7/500
233/233 [==============================] - 5s 20ms/step - loss: 0.0024 - val_loss: 0.0127
Epoch 8/500
233/233 [==============================] - 5s 20ms/step - loss: 0.0024 - val_loss: 0.0122
Epoch 9/500
233/233 [==============================] - 5s 21ms/step - loss: 0.0023 - val_loss: 0.0120
Epoch 10/500
233/233 [==============================] - 5s 21ms/step - loss: 0.00

In [168]:
# loss = pd.DataFrame(model1.history.history)


In [169]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('train_loss', fontsize=15)
# plt.ylabel('Loss', fontsize=15)
# plt.xlabel('Epoch', fontsize=15)
# plt.legend(['train', 'val'], loc='upper right')
# plt.xticks(fontsize=15)
# plt.yticks(fontsize=15)
# plt.show()

In [170]:
var = 'x_y_xy';

In [171]:
# model_json = model1.to_json()
# with open('Saved_model/train_model2_ref_'+var+'.json', 'w') as json_file:
#     json_file.write(model_json)

# #serialize weights to HDF5 file
# model1.save_weights('Saved_model/train_model2_ref_'+var+'.h5')


In [172]:
#Saving train model with different optimizer.
model_json = model1.to_json()
with open('Model/Model2/SGD_'+var+'.json', 'w') as json_file:
    json_file.write(model_json)

#serialize weights to HDF5 file
model1.save_weights('Model/Model2/SGD_'+var+'.h5')


In [173]:
# # load json and create model
# json_file = open('Saved_model/train_model2_ref_'+var+'.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json);

# #load weights into new model
# loaded_model.load_weights('Saved_model/train_model2_ref_'+var+'.h5') 

In [174]:
#Loding model with different optimizer.
# load json and create model
json_file = open('Model/Model2/SGD_'+var+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json);

#load weights into new model
loaded_model.load_weights('Model/Model2/SGD_'+var+'.h5') 

In [175]:
#Stress calculation at Gauss points
stress = np.zeros((nel, ngp2d*ngp2d, 3))
strains = np.zeros((nel, ngp2d*ngp2d, 3))

for i in range(nel):
    stress_i_g = np.zeros((ngp2d*ngp2d, 3))
    strains_i_g = np.zeros((ngp2d*ngp2d, 3))

    stress_i_g, strains_i_g = stress_gauss.get_element_stress(
        i, ngp2d, el_type_q4, connect, coord, u, C)

    stress[i][:][:] = stress_i_g
    strains[i][:][:] = strains_i_g.reshape((1, 3))


In [176]:
# calcuation of gauss coordinates.
gauss_coords = np.zeros((nel, ngp2d*ngp2d, 2))
gp, weights = quadrature.quadrature(ngp2d)
for i in range(nel):
    node = connect[i, :]
    vertex_coord = coord[node, :].reshape(-1)
    gauss_coords[i][:][:] = gauss_pt_coord.gauss_pts(
        ngp2d, vertex_coord, gp, el_type)
gauss_coords = gauss_coords.reshape(gauss_coords.shape[0], -1)


In [177]:
#creation of patches for spr_stress;
patch, n_patches, int_nodes = patch_n_int_nodes.patch_n_int_nodes(ms)


In [178]:
#spr_stress STRESS Calculations
stress_spr = spr_stress.spr(gauss_coords, coord, connect, stress,
                            int_nodes, n_patches, patch, ms)
# stress_spr


In [179]:
#Directly calculated stress
stress_dc, strain_dc = stress_nodes_dc.stress_dc(
    connect, coord, u, nel, el_type, C)


In [180]:
# coord
# np.random.seed(1);
# pts = np.random.choice(np.arange(36),5);
# coord_test = coord[pts];
# coord_test
# stress_spr
total_nodes = (ms+1)**2


In [181]:
#outter points in
sp = []
# print(ms)
temp = ms+1
for i in range(ms-1):
    sp.append(temp)
    sp.append(temp+ms)
    temp = temp+ms+1
for i in range(ms+1):
    sp.append(temp)
    temp = temp+1

for i in range(ms+1):
    sp.append(i)

sp = sorted(sp)
# print(sp);


In [182]:
coord_corner = coord[sp]
# coord_corner


In [183]:
scaled_test_corner = scaler.transform(coord_corner)
# scaled_test_corner


/home/krishna/Desktop/btp_499_code/btp_env/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [184]:
ref_stress = pd.read_csv('Ref/ref_stress_for_ms_5_from_320.csv')
ref_stress = ref_stress.loc[:, ~ref_stress.columns.str.contains('^Unnamed')]
ref_stress = ref_stress.to_numpy()
# ref_stress


In [185]:
# a ---->reference stress
a = ref_stress.T
print(len(a))


3


In [186]:
# a--->plane stress calculated from spr
b = stress_spr.T
print(len(b))


3


In [187]:
err_spr_abs = np.abs((a-b))
err_spr_perc = np.abs((a-b)/a)
err_dc_abs = np.abs((a-stress_dc.T))
err_dc_perc = np.abs((a-stress_dc.T)/a)
# err_spr_abs
# err_dc_abs


In [188]:
test = pd.DataFrame(scaled_test, columns=['a', 'b'])
# test


In [189]:
# c --->stress calculated using ANN trained model
c = loaded_model.predict(test)
len(c)


2/2 [==============================] - 1s 39ms/step


36

In [190]:
err_ann_abs = np.abs((a-c.T))
err_ann_perc = np.abs((a-c.T)/a)
# err_ann_abs


In [191]:

alll = np.concatenate([coord,
                       a.reshape(a.T.shape[0], 3),
                       b.reshape(b.T.shape[0], 3),
                       c.reshape(c.shape[0], 3),
                       err_spr_abs.reshape(err_spr_abs.T.shape[0], 3),
                       err_ann_abs.reshape(err_ann_abs.T.shape[0], 3),
                       err_dc_abs.reshape(err_dc_abs.T.shape[0], 3),
                       err_spr_perc.reshape(err_spr_perc.T.shape[0], 3),
                       err_ann_perc.reshape(err_ann_perc.T.shape[0], 3),
                       err_dc_perc.reshape(err_dc_perc.T.shape[0], 3)
                       ],
                      axis=1)
# alll


In [192]:
tdf = pd.DataFrame(alll, columns=['x',
                                  'y',
                                  'ref_stress_x',
                                  'ref_stress_y',
                                  'ref_stress_xy',
                                  'spr_stress_x',
                                  'spr_stress_y',
                                  'spr_stress_xy',
                                  'ann_stress_x',
                                  'ann_stress_y',
                                  'ann_stress_xy',
                                  'err_spr_abs_x',
                                  'err_spr_abs_y',
                                  'err_spr_abs_xy',
                                  'err_ann_abs_x',
                                  'err_ann_abs_y',
                                  'err_ann_abs_xy',
                                  'err_dc_abs_x',
                                  'err_dc_abs_y',
                                  'err_dc_abs_xy',
                                  'err_spr_perc_x',
                                  'err_spr_perc_y',
                                  'err_spr_perc_xy',
                                  'err_ann_perc_x',
                                  'err_ann_perc_y',
                                  'err_ann_perc_xy',
                                  'err_dc_perc_x',
                                  'err_dc_perc_y',
                                  'err_dc_perc_xy'
                                  ])
# print(tdf.head());


In [193]:
tdf.to_csv('mult_e_data/ms5/all_matlab_train_model2_ref_'+var+'.csv')
tdf.head()


,x,y,ref_stress_x,ref_stress_y,ref_stress_xy,spr_stress_x,spr_stress_y,spr_stress_xy,ann_stress_x,ann_stress_y,...,err_dc_abs_xy,err_spr_perc_x,err_spr_perc_y,err_spr_perc_xy,err_ann_perc_x,err_ann_perc_y,err_ann_perc_xy,err_dc_perc_x,err_dc_perc_y,err_dc_perc_xy
0,0.0,0.0,0.020590,0.066856,0.114837,0.063443,0.059761,0.057295,0.056644,0.032010,...,0.003450,2.081206,0.106128,0.501073,1.751031,0.206221,0.020695,2.244944,0.033750,0.030039
1,9.6,8.8,0.142259,0.142736,0.116694,0.060733,0.062001,0.065805,0.080643,0.041843,...,0.285664,0.573084,0.565624,0.436090,0.059730,0.205023,0.399891,0.489825,0.133649,2.447965
2,19.2,17.6,0.073042,0.093437,0.099048,0.063212,0.086114,0.096941,0.112460,0.054878,...,0.017992,0.134571,0.078366,0.021269,0.111683,0.073298,0.062605,0.192864,0.120837,0.181649
3,28.8,26.4,0.094280,0.076036,0.014692,0.093381,0.065664,0.014115,0.133762,0.064998,...,0.006585,0.009532,0.136410,0.039297,0.022444,0.054796,1.756047,0.152581,0.531915,0.448163
4,38.4,35.2,0.080207,0.072637,0.051343,0.027299,0.068231,0.050842,0.113471,0.065836,...,0.007476,0.659641,0.060657,0.009764,0.141788,0.114334,0.029884,0.028527,0.229736,0.145609


In [194]:
tdf['err_spr_perc_x'].mean()


0.6860461513578601

In [195]:
tdf['err_spr_perc_y'].mean()


0.32947246179473

In [196]:
tdf['err_spr_perc_xy'].mean()


0.5658925933085241

In [197]:
tdf['err_ann_perc_x'].mean()


0.7493842927119523

In [198]:
tdf['err_ann_perc_y'].mean()


0.7812233367437786

In [199]:
tdf['err_ann_perc_xy'].mean()

0.7433034974780829